### Masked Self-Attention中Mask是怎么实现的？

- 原始的注意力分数计算公式为：
$$\begin{align*}
X &\in \mathbb{R}^{\text{seq\_len} \times d_{model}} \\
Q &= XW_Q, W_Q \in \mathbb{R}^{d_{model} \times d_k} \\
K &= XW_K, W_K \in \mathbb{R}^{d_{model} \times d_k} \\
V &= XW_V, W_V \in \mathbb{R}^{d_{model} \times d_v} \\
S &= \frac{QK^T}{\sqrt{d_k}}, S \in \mathbb{R}^{\text{seq\_len} \times \text{seq\_len}} \\
Attention(Q, K, V) &= softmax(S) V, Attention(Q, K, V) \in \mathbb{R}^{\text{seq\_len} \times d_v}
\end{align*}$$
- $d_v$一般等于$d_{model}$，保证输出Attension的维度和输入X的维度一致（便于后续的注意力或者FeedForward层）


- 如果是Masked Self-Attention, 我们需要加上以一个mask掩码矩阵$M \in \mathbb{R}^{\text{seq\_len} \times \text{seq\_len}}$，来屏蔽掉未来时刻的信息:
$$\begin{align*}
S &= S + M \\
M &= \begin{bmatrix}
0 & -inf & -inf & -inf \\
0 & 0 & -inf & -inf \\
\ldots & \ldots & \ldots & \ldots \\
0 & 0 & 0 & -inf \\
\end{bmatrix}
\end{align*}$$

- 然后再进行softmax计算，并乘以V:
$$\begin{align*}
Attention(Q, K, V) = softmax(S) V
\end{align*}$$

- 其中，每一行的计算都是可以并行的。

## Cross-Attention中哪些信息来自别的块？

- 在传统的Encoder-Decoder架构中。
  - Cross-Attention的Query来自Decoder的上一层(Masked Self-Attention)输出，
  - 而Key和Value则来自Encoder的输出。(memory[batch_size, src_len, d_model]/encoder_output是整个句子通过Encoder层后的输出)
    - key = memory * W_k
    - value = memory * W_v
    - W_K和W_V是cross atten层可训练的权重矩阵。
  - 还有src_mask(源序列的mask), 用于处理源序列中的填充部分(padding)。

## 残差连接的作用?

- 残差连接:
  1. 缓解梯度消失：让梯度直接回传
     - $y=F(x)$, 导数$d\text{loss}/dx = d\text{loss}/dy \times dF(x)/dx$
     - $y=x + F(x)$, 导数$d\text{loss}/dx = d\text{loss}/dy \times (1 + dF(x)/dx)$
     - 即使$dF(x)/dx$趋于0, 仍然保证$d\text{loss}/dy$这部分
     - 让100层网络的训练像10层网络一样稳定
     - 底层参数也能获得有效的梯度更新
  2. 保持信息流动：保留原始输入信息
     - 每层输出 = 原始输入 + 学习到的变化, output = input + learned_change
     - 恒等映射捷径：原始输入 x 直接传递到输出
     - 选择性学习：网络只需要学习相对于输入的"残差" F(x)
     - 信息无损传递：重要特征不会在深层中丢失
  3. 训练稳定性：使深层网络更容易训练
     - 深层网络容易出现的问题:
       - Loss震荡剧烈
       - 需要精心调参和学习率策略
       - 对初始化敏感
     - 训练过程更加稳定和平滑
     - 对超参数的敏感性降低
     - 可以使用更大的学习率加速收敛